In [1]:
# This makes a first contrast curve for Altair

# created 2018 Sept. 20 by E.S.

## SECTION TO INITIALIZE

In [1]:
# import stuff

import urllib
import numpy as np
import matplotlib.pyplot as plt

import PynPoint

from PynPoint import Pypeline
from PynPoint.IOmodules.Hdf5Reading import Hdf5ReadingModule
from PynPoint.IOmodules.FitsWriting import FitsWritingModule
from PynPoint.IOmodules.FitsReading import FitsReadingModule
from PynPoint.IOmodules.TextReading import ParangReadingModule
from PynPoint.ProcessingModules import PSFpreparationModule, \
                                       PcaPsfSubtractionModule, \
                                       ContrastCurveModule, \
                                       FluxAndPosition, \
                                    StackingAndSubsampling
from PynPoint.ProcessingModules.StackingAndSubsampling import DerotateAndStackModule
from PynPoint.ProcessingModules.FluxAndPosition import FakePlanetModule
#from PynPoint.Util import AnalysisTools

In [2]:
# define workspaces and initialize Pypeline

working_place = "./pynpoint_experimentation_altair/working_place/"
input_place = "./pynpoint_experimentation_altair/input_place/"
output_place = "./pynpoint_experimentation_altair/output_place/"

pipeline = Pypeline(working_place_in=working_place,
                    input_place_in=input_place,
                    output_place_in=output_place)

# now a *.ini file has been generated (this includes the PIXSCALE), if no pre-existing one was there

Initiating PynPoint... [DONE]


In [3]:
## IF THE *INI FILE WAS NEWLY GENERATED, EDIT THE *INI FILE TO SET
## PIXSCALE = 0.0107

## SECTION TO READ IN DATA

In [4]:
# read in science FITS files 
# (should have PARANG=0 in all headers, so as to keep the PSF in the same original orientation on the array; 
# we need to wait to correct for PARANG later)

read_science = FitsReadingModule(name_in="read_science",
                                 input_dir=None,
                                 image_tag="science",
                                 check=True)

pipeline.add_module(read_science)

In [5]:
# confirm PIXSCALE change
'''
pixscale_config = pipeline.get_attribute("config", "PIXSCALE")
print("Plate scale for /config/ is "+pixscale_config)
pixscale_sci = pipeline.get_attribute("science", "PIXSCALE")
print("Plate scale for /science/ is "+pixscale_sci)
'''

'\npixscale_config = pipeline.get_attribute("config", "PIXSCALE")\nprint("Plate scale for /config/ is "+pixscale_config)\npixscale_sci = pipeline.get_attribute("science", "PIXSCALE")\nprint("Plate scale for /science/ is "+pixscale_sci)\n'

In [6]:
# read in PSF reference FITS files (i.e., unsaturated frames)
# (these should also not have any PARANG)

read_ref_psf = FitsReadingModule(name_in="read_ref_psf",
                                 input_dir=input_place+'ref_psf/',
                                 image_tag="ref_psf",
                                 check=True)

pipeline.add_module(read_ref_psf)

In [6]:
## SECTION TO PROCESS DATA

In [6]:
## PIPELINE CHANNEL 1: IMAGE RESIDUALS

# generate PCA basis from unsaturated frames and
# do PCA PSF subtraction of the saturated frames

pca_pca_subt = PcaPsfSubtractionModule(pca_numbers=(100, ),
                                       name_in="pca",
                                       images_in_tag="science",
                                       reference_in_tag="ref_psf",
                                       res_mean_tag="mean_residuals",
                                       res_median_tag="median_residuals",
                                       res_arr_out_tag="all_resids",
                                       res_rot_mean_clip_tag="resid_rot",
                                       basis_out_tag="pca_components",
                                       subtract_mean=True,
                                       verbose=True)

pipeline.add_module(pca_pca_subt)

# note:
# images_in_tag: science images
# reference_in_tag: reference images, which COULD be the science images

In [7]:
## PIPELINE CHANNEL 1: IMAGE RESIDUALS

# now assign physical, non-zero parallactic angles to science frames

parang = ParangReadingModule("parang_vals.csv",
                             name_in="parang",
                             input_dir=input_place,
                             data_tag="all_resids100",
                             overwrite=True)

pipeline.add_module(parang)

## PIPELINE CHANNEL TEST: INJECT FAKE PLANET AND SEE IF THE PLANET PSF ITSELF GETS ROTATED

In [7]:
## PIPELINE CHANNEL TEST: INJECT FAKE PLANET AND SEE IF THE PLANET PSF ITSELF GETS ROTATED

## THERE MIGHT BE 2 WAYS OF DOING THIS: WITH FAKE_PLANET OR WITH FAKEPLANETMODULE 

# test 1: images all have set PA=0
'''
fake_planet_injection = AnalysisTools.fake_planet(images="read_science",
                                    psf="ref_psf",
                                    parang="parang",
                                    position=(15,0),
                                    magnitude=0,
                                    psf_scaling=1,
                                    interpolation="spline")
'''

fake_planet_injection = FakePlanetModule(position=(0.25,0),
                                         magnitude=0,
                                         psf_scaling=1,
                                         interpolation="spline",
                                         name_in="fake_planet_injection",
                                         image_in_tag="science",
                                         psf_in_tag="ref_psf",
                                         image_out_tag="test_planet_inject")

pipeline.add_module(fake_planet_injection)


# write out result

write_fake_injection = FitsWritingModule(file_name="fake_injection_tests.fits",
                              name_in="write_fake_injection",
                              output_dir=output_place,
                              data_tag="test_planet_inject")

pipeline.add_module(write_fake_injection)


pipeline.run()

Validating Pypeline... [DONE]


/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute DIT (=ESO DET DIT) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute LATITUDE (=ESO TEL GEOLAT) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute LONGITUDE (=ESO TEL GEOLON) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:207: UserWarning: Non-static attribute EXP_NO (=ESO DET EXP NO) not found in the FITS header.
  "header." % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:207: UserWarning: Non-static attribute ND

Running FitsReadingModule... [DONE]
Running FitsReadingModule... [DONE]
Running FakePlanetModule... [DONE]


/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute INSTRUMENT (=INSTRUME) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:207: UserWarning: Non-static attribute DATE (=DATE-OBS) not found in the FITS header.
  "header." % (item[0], fitskey))


Running FitsWritingModule... [DONE]


## PIPELINE CHANNEL 1: IMAGE RESIDUALS

In [8]:
## PIPELINE CHANNEL 1: IMAGE RESIDUALS

# derotate and median the science images based on physical 
# parallactic angles

derotate_sci = DerotateAndStackModule(name_in="derotate_sci",
                                        image_in_tag="all_resids100",
                                        image_out_tag="derotated_psf_subted",
                                        derotate=True,
                                        stack="median",
                                        extra_rot=0.0)

pipeline.add_module(derotate_sci)

In [7]:
## PIPELINE CHANNEL 2: MAKE CONTRAST CURVE

# make a contrast curve 
# (N.b. this does not separately require PcaPsfSubtractionModule)

'''
cent_size: mask radius
'''

contrast_curve = ContrastCurveModule(name_in="contrast_curve",
                            image_in_tag="science",
                            psf_in_tag="ref_psf",
                            contrast_out_tag="contrast_landscape",
                            pca_out_tag="pca_resids",
                            pca_number=20,
                            psf_scaling=1,
                            separation=(0.25, 0.26, 0.1), 
                            angle=(0.0, 360.0, 360.0), 
                            magnitude=(7.5, 1.0),
                            cent_size=None)

pipeline.add_module(contrast_curve)

In [ ]:
## SECTION TO WRITE OUT DATA

In [9]:
# write out PCA basis set

write_pca_comp = FitsWritingModule(file_name="pca_components.fits",
                              name_in="write_pca_comp",
                              output_dir=output_place,
                              data_tag="pca_components")

pipeline.add_module(write_pca_comp)

In [10]:
# write out PCA-based PSF-subtracted residuals out

write_mean_resids = FitsWritingModule(file_name="junk_mean_residuals.fits",
                              name_in="write_mean_resids",
                              output_dir=output_place,
                              data_tag="mean_residuals")

pipeline.add_module(write_mean_resids)

In [11]:
# write out PCA-based PSF-subtracted residuals out

write_median_resids = FitsWritingModule(file_name="junk_median_residuals.fits",
                              name_in="write_median_resids",
                              output_dir=output_place,
                              data_tag="median_residuals")

pipeline.add_module(write_median_resids)

In [12]:
# write out PCA-based PSF-subtracted residuals out (is this piece working??)

'''
write_all_resids5 = FitsWritingModule(file_name="junk_all_residuals5.fits",
                              name_in="write_all_resids5",
                              output_dir=output_place,
                              data_tag="all_resids5")

pipeline.add_module(write_all_resids5)
'''

'\nwrite_all_resids5 = FitsWritingModule(file_name="junk_all_residuals5.fits",\n                              name_in="write_all_resids5",\n                              output_dir=output_place,\n                              data_tag="all_resids5")\n\npipeline.add_module(write_all_resids5)\n'

In [13]:
# write out PCA-based PSF-subtracted residuals out

write_all_resids100 = FitsWritingModule(file_name="junk_all_residuals100.fits",
                              name_in="write_all_resids100",
                              output_dir=output_place,
                              data_tag="all_resids100")

pipeline.add_module(write_all_resids100)

In [14]:
# write out derotated, PSF-subtracted images

write_all_derotated_psf_subted = FitsWritingModule(file_name="junk_all_derotated_residuals.fits",
                                                   name_in="write_all_derotated_psf_subted",
                                                   output_dir=output_place,
                                                   data_tag="derotated_psf_subted")

pipeline.add_module(write_all_derotated_psf_subted)

In [15]:
## might insert PSF Preparation here later, to upsample and normalize (but not mask! let contrast curve module do that)

In [16]:
pipeline.run()

Validating Pypeline... [DONE]


/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute DIT (=ESO DET DIT) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute LATITUDE (=ESO TEL GEOLAT) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute LONGITUDE (=ESO TEL GEOLON) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:207: UserWarning: Non-static attribute EXP_NO (=ESO DET EXP NO) not found in the FITS header.
  "header." % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:207: UserWarning: Non-static attribute ND

Running FitsReadingModule... [DONE]


/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:172: UserWarning: Static attribute INSTRUMENT (=INSTRUME) not found in the FITS header.
  % (item[0], fitskey))
/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/IOmodules/FitsReading.py:207: UserWarning: Non-static attribute DATE (=DATE-OBS) not found in the FITS header.
  "header." % (item[0], fitskey))


Running FitsReadingModule... [DONE]
Constructing PSF model... [DONE]
Creating residuals... [DONE]
Running ParangReadingModule... [DONE]
Running DerotateAndStackModule... [DONE]
Running FitsWritingModule... [DONE]
Running FitsWritingModule... [DONE]
Running FitsWritingModule... [DONE]
Running FitsWritingModule... [DONE]
Running FitsWritingModule... [DONE]


In [10]:
test = pipeline.get_attribute("header_all_resids100", "PARANG")

KeyError: "Can't open attribute (Can't locate attribute: 'parang')"

In [9]:
test2 = pipeline.get_data(tag="/header_all_resids100/PARANG")

In [10]:
print(test2)

[-37.722728  -37.716388  -37.710286  ...  -1.8834054  -1.8714235
  -1.8714235]


In [ ]:
##########################################################################

In [10]:
contrast_curve_results = pipeline.get_data("contrast_landscape")

In [11]:
# contrast curve

# [0]: separation
# [1]: azimuthally averaged contrast limits
# [2]: the azimuthal variance of the contrast limits
# [3]: threshold of the false positive fraction associated with sigma

contrast_curve_results

array([[2.50000000e-01,            nan,            nan, 1.21477003e-04]])

In [ ]:
# make plots

f, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(contrast_curve_results[:,0],contrast_curve_results[:,1])
ax1.set_title('Azim. averaged contrast limit')
ax1.set_xlabel('Radius (asec)')
ax2.set_ylabel('del_mag')
ax2.plot(contrast_curve_results[:,0],contrast_curve_results[:,3])
ax2.set_title('Threshold of FPF')
ax2.set_xlabel('Radius (asec)')
f.savefig('test.png')

In [ ]:
##########################

In [5]:
# inject fake planets

'''
inject = FluxAndPosition.FakePlanetModule(position=(0.46,0),
                          magnitude=0,
                          psf_scaling=1,
                          name_in="inject",
                          image_in_tag="stack",
                          psf_in_tag="model_psf",
                          image_out_tag="fake_planet_output")

pipeline.add_module(inject)
'''

'\ninject = FluxAndPosition.FakePlanetModule(position=(0.46,0),\n                          magnitude=0,\n                          psf_scaling=1,\n                          name_in="inject",\n                          image_in_tag="stack",\n                          psf_in_tag="model_psf",\n                          image_out_tag="fake_planet_output")\n\npipeline.add_module(inject)\n'

In [20]:
# prepare psf


prep_fake_planet = PSFpreparationModule(name_in="prep_fake_planet",
                            image_in_tag="stack",
                            image_out_tag="prep",
                            image_mask_out_tag=None,
                            mask_out_tag=None,
                            norm=False,
                            resize=None,
                            cent_size=0.15,
                            edge_size=1.1)

pipeline.add_module(prep_fake_planet)

In [21]:
pipeline.run()

# after running the pipeline, hdf5 attributes like PIXSCALE for groups
# other than /config/ should be available

Validating Pypeline... [DONE]
Running Hdf5ReadingModule... [DONE]
Running FitsReadingModule... [DONE]
Running PSFpreparationModule... [DONE]


In [22]:
pixscale_config = pipeline.get_attribute("config", "PIXSCALE")
print(pixscale_config)
pixscale_prep = pipeline.get_attribute("prep", "PIXSCALE")
print(pixscale_prep)

0.0107
0.027


In [7]:
print(pixscale_config)

0.027


In [8]:
# write out prepped psf

'''
# checks HDF5 file
write_prep = FitsWritingModule(file_name="junk_prep.fits",
                              name_in="write_prep",
                              output_dir=output_place,
                              data_tag="prep")

#pipeline.add_module(write_prep)
'''

'\n# checks HDF5 file\nwrite_prep = FitsWritingModule(file_name="junk_prep.fits",\n                              name_in="write_prep",\n                              output_dir=output_place,\n                              data_tag="prep")\n\n#pipeline.add_module(write_prep)\n'

In [7]:
# do PCA PSF subtraction

'''
pca = PcaPsfSubtractionModule(pca_numbers=(5, ),
                              name_in="pca",
                              images_in_tag="fake_planet_output",
                              reference_in_tag="fake_planet_output",
                              res_mean_tag="mean_residuals",
                              res_median_tag="median_residuals",
                              res_arr_out_tag="all_resids",
                              res_rot_mean_clip_tag="resid_rot",
                              verbose=True)

pipeline.add_module(pca)

# note:
# images_in_tag: science images
# reference_in_tag: reference images, which COULD be the science images
'''

In [7]:
## THIS IS TEST ONLY ## do PCA PSF subtraction

'''
pca = PcaPsfSubtractionModule(pca_numbers=(5, ),
                              name_in="pca",
                              images_in_tag="read_science",
                              reference_in_tag="read_science",
                              res_mean_tag="mean_residuals",
                              res_median_tag="median_residuals",
                              res_arr_out_tag="all_resids",
                              res_rot_mean_clip_tag="resid_rot",
                              verbose=True)

pipeline.add_module(pca)
'''
# note:
# images_in_tag: science images
# reference_in_tag: reference images, which COULD be the science images

In [8]:
# write out outputs from PCA PSF subtraction

'''
# checks HDF5 file
read_test1 = FitsWritingModule(file_name="junk_mean_residuals.fits",
                              name_in="read_test1",
                              output_dir=output_place,
                              data_tag="mean_residuals")

pipeline.add_module(read_test1)
'''

In [9]:
# write out outputs from PCA PSF subtraction

'''
# checks HDF5 file
read_test2 = FitsWritingModule(file_name="junk_median_residuals.fits",
                              name_in="read_test2",
                              output_dir=output_place,
                              data_tag="median_residuals")

pipeline.add_module(read_test2)
'''

In [10]:
# write out outputs from PCA PSF subtraction

'''
# checks HDF5 file
read_test4 = FitsWritingModule(file_name="junk_resid_rot.fits",
                              name_in="read_test4",
                              output_dir=output_place,
                              data_tag="resid_rot")

pipeline.add_module(read_test4)
'''

In [6]:
# make a contrast curve

'''
cent_size: mask radius
'''

contrast_curve = ContrastCurveModule(name_in="contrast_curve",
                            image_in_tag="prep",
                            psf_in_tag="model_psf",
                            contrast_out_tag="contrast_landscape",
                            pca_out_tag="pca_resids",
                            pca_number=20,
                            psf_scaling=1,
                            separation=(0.1, 1.0, 0.1), 
                            angle=(0.0, 360.0, 60.0), 
                            magnitude=(7.5, 1.0),
                            cent_size=None)

pipeline.add_module(contrast_curve)

In [9]:
pipeline.run()

Validating Pypeline... [DONE]


AssertionError: Error: Input file does not exist. Input requested: BetaPic_NACO_Mp.hdf5

In [11]:
contrast_curve_results = pipeline.get_data("contrast_landscape")
#residuals = pipeline.get_data("residuals")
#pixscale = pipeline.get_attribute("stack", "PIXSCALE")

#size = pixscale*np.size(residuals, 1)/2.

In [12]:
# contrast curve

# [0]: separation
# [1]: azimuthally averaged contrast limits
# [2]: the azimuthal variance of the contrast limits
# [3]: threshold of the false positive fraction associated with sigma

contrast_curve_results

array([[1.00000000e-01,            nan,            nan, 3.74521694e-03],
       [2.00000000e-01,            nan,            nan, 2.68666801e-04],
       [3.00000000e-01, 5.07657121e+00, 1.80844893e-01, 6.54338490e-05],
       [4.00000000e-01, 4.73267286e+00, 8.75829719e-02, 2.33216658e-05],
       [5.00000000e-01, 4.57947726e+00, 1.54244613e-02, 1.26831579e-05],
       [6.00000000e-01, 6.79627759e+00, 7.58776386e-02, 8.02141316e-06],
       [7.00000000e-01, 7.15955594e+00, 8.30756762e-02, 5.60807982e-06],
       [8.00000000e-01, 7.48905131e+00, 4.07430502e-02, 4.03083259e-06],
       [9.00000000e-01, 7.51861472e+00, 1.13346032e-02, 3.20495930e-06]])

In [20]:


f, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(contrast_curve_results[:,0],contrast_curve_results[:,1])
ax1.set_title('Azim. averaged contrast limit')
ax1.set_xlabel('Radius (asec)')
ax2.set_ylabel('del_mag')
ax2.plot(contrast_curve_results[:,0],contrast_curve_results[:,3])
ax2.set_title('Threshold of FPF')
ax2.set_xlabel('Radius (asec)')
f.savefig('test.png')

In [19]:
plt.imshow(residuals[0, ], origin='lower', extent=[size, -size, -size, size])
plt.title("beta Pic b - NACO M' - mean residuals")
plt.xlabel('R.A. offset [arcsec]', fontsize=12)
plt.ylabel('Dec. offset [arcsec]', fontsize=12)
plt.colorbar()
#plt.show()
plt.savefig(output_place+"residuals.png")

NameError: name 'residuals' is not defined

In [13]:
pipeline

In [19]:
np.size(residuals)

NameError: name 'residuals' is not defined

In [10]:
pipeline.add_module(writefits)

/home/unasemaje/anaconda3/envs/py27/lib/python2.7/site-packages/PynPoint/Core/Pypeline.py:257: UserWarning: Processing module names need to be unique. Overwriting module 'fits_write'.
  % module.name)


In [11]:
writefits.run()

Running FitsWritingModule... [DONE]
